# Import data

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect

In [3]:
engine = create_engine('sqlite:///olympics.db')

In [4]:
inspector = inspect(engine)
tables = inspector.get_table_names()
tables

['city',
 'competitor_event',
 'event',
 'games',
 'games_city',
 'games_competitor',
 'medal',
 'noc_region',
 'person',
 'person_region',
 'sport']

# Exercise 1: Complex Subquery Analysis

In [5]:
# Task 1: Find the average age of competitors who have won at least one medal,
#  grouped by the type of medal they won. Use a correlated subquery to achieve this

df11 = pd.read_sql('''
                SELECT m.medal_name AS medal, AVG(gc.age) AS average_age
                FROM games_competitor gc
                JOIN competitor_event ce ON gc.id = ce.competitor_id
                JOIN medal m ON ce.medal_id = m.id
                WHERE ce.medal_id IS NOT NULL
                GROUP BY m.medal_name;
                   ''', engine)

df11


,medal,average_age
0,Bronze,25.881749
1,Gold,25.915993
2,NA,25.494186
3,Silver,26.006659


In [6]:
# Task 1: Find the average age of competitors who have won at least one medal,
#  grouped by the type of medal they won. Use a correlated subquery to achieve this

df11 = pd.read_sql("""
SELECT m.medal_name AS medal,
       (SELECT AVG(gc.age)
        FROM games_competitor gc
        JOIN competitor_event ce ON gc.id = ce.competitor_id
        WHERE ce.medal_id = m.id) AS average_age
FROM medal m
WHERE m.id IN (
    SELECT DISTINCT medal_id
    FROM competitor_event
    WHERE medal_id IS NOT NULL
);
""", engine)

df11


,medal,average_age
0,Gold,25.915993
1,Silver,26.006659
2,Bronze,25.881749
3,NA,25.494186


In [7]:
# Task 2: Identify the top 5 regions with the highest number of unique competitors who have participated 
# in more than 3 different events. Use nested subqueries to filter and aggregate the data.

df12 = pd.read_sql("""
SELECT nr.region_name AS region, COUNT(DISTINCT pr.person_id) AS unique_competitors
FROM person_region pr
JOIN noc_region nr ON pr.region_id = nr.id
WHERE pr.person_id IN (
    SELECT ce.competitor_id
    FROM competitor_event ce
    GROUP BY ce.competitor_id
    HAVING COUNT(DISTINCT ce.event_id) > 3
)
GROUP BY nr.region_name
ORDER BY unique_competitors DESC
LIMIT 5;
""", engine)

df12


,region,unique_competitors
0,USA,368
1,UK,233
2,Germany,198
3,France,198
4,Canada,194


In [13]:
# Task 3: Create a temporary table to store the total number of medals won by each competitor
#  and filter to show only those who have won more than 2 medals. Use subqueries to aggregate the data.


# Step 1: Start a transaction using `.begin()`
with engine.begin() as connection:
    # Drop the temporary table if it exists
    connection.execute(text("DROP TABLE IF EXISTS competitor_medals;"))

    # Create the temporary table within the transaction
    connection.execute(text("""
        CREATE TEMPORARY TABLE competitor_medals AS
        SELECT ce.competitor_id, COUNT(ce.medal_id) AS total_medals
        FROM competitor_event ce
        WHERE ce.medal_id IS NOT NULL
        GROUP BY ce.competitor_id;
    """))

# Step 2: Use read_sql to fetch results from the temporary table
df12 = pd.read_sql("""
    SELECT p.full_name AS competitor_name, cm.total_medals
    FROM competitor_medals cm
    JOIN person p ON cm.competitor_id = p.id
    WHERE cm.total_medals > 2;
    """, engine)

# Display the DataFrame
df12



,competitor_name,total_medals
0,Cornelia 'Cor'' Aalten (-Strannood)',4
1,Antti Sami Aalto,4
2,Einar Ferdinand 'Einari'' Aalto',4
3,Jorma Ilmari Aalto,4
4,Nils Egil Aaness,8
...,...,...
11189,"Ronald Ferdinand ""Ron"""" Zwerver""",6
11190,Marc Zwiebler,6
11191,Yevgeny Aleksandrovich Zykov,4
11192,Fernando scar Zylberberg,4


In [14]:
# Task 4: Use a subquery within a DELETE statement to remove records of competitors
#  who have not won any medals from a temporary table created for analysis.

with engine.begin() as conn:
    # Create a temporary table for competitor analysis
    conn.execute(text("""
        CREATE TEMPORARY TABLE competitor_analysis AS
        SELECT ce.competitor_id, COUNT(ce.medal_id) AS total_medals
        FROM competitor_event ce
        GROUP BY ce.competitor_id;
    """))

    # Delete competitors with no medals using a subquery
    conn.execute(text("""
        DELETE FROM competitor_analysis
        WHERE competitor_id IN (
            SELECT competitor_id
            FROM competitor_analysis
            WHERE total_medals = 0
        );
    """))

# Fetch the remaining competitors to verify
query = "SELECT * FROM competitor_analysis;"
df = pd.read_sql(query, engine)
print(df)

        competitor_id  total_medals
0                   1             1
1                   2             1
2                   3             1
3                   4             1
4                   5             2
...               ...           ...
180247         180248             1
180248         180249             1
180249         180250             2
180250         180251             1
180251         180252             1

[180252 rows x 2 columns]


# Exercise 2: Advanced Data Manipulation and Optimization

In [15]:
# Task 1: Update the heights of competitors based on the average height of competitors 
# from the same region. Use a correlated subquery within the UPDATE statement.

with engine.begin() as conn:
    # Update competitor heights based on the average height from the same region
    conn.execute(text("""
        UPDATE person
        SET height = (
            SELECT AVG(p2.height)
            FROM person p2
            JOIN person_region pr2 ON p2.id = pr2.person_id
            WHERE pr2.region_id = (
                SELECT pr.region_id
                FROM person_region pr
                WHERE pr.person_id = person.id
            )
        )
        WHERE height IS NULL;
    """))

# Query to verify the updated heights
query = """
    SELECT p.id, p.full_name, p.height, pr.region_id
    FROM person p
    JOIN person_region pr ON p.id = pr.person_id
    LIMIT 10;
"""
df = pd.read_sql(query, engine)
print(df)

   id                             full_name  height  region_id
0   1                             A Dijiang     180         42
1   2                              A Lamusi     170         42
2   3                   Gunnar Nielsen Aaby       0         56
3   4                  Edgar Lindenau Aabye       0         56
4   5              Christine Jacoba Aaftink     185        146
5   6                       Per Knut Aaland     188        217
6   7                          John Aalberg     183        217
7   8  Cornelia 'Cor'' Aalten (-Strannood)'     168        146
8   9                      Antti Sami Aalto     186         69
9  10      Einar Ferdinand 'Einari'' Aalto'       0         69


In [16]:
# Task 2: Insert new records into a temporary table for competitors who participated
#  in more than one event in the same games and list their total number of events participated. Use nested subqueries for filtering.

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TEMP TABLE multi_event_competitors AS
        SELECT
            gc.person_id,
            gc.games_id,
            COUNT(ce.event_id) AS total_events
        FROM games_competitor AS gc
        JOIN competitor_event AS ce ON gc.id = ce.competitor_id
        GROUP BY gc.person_id, gc.games_id
        HAVING COUNT(ce.event_id) > 1;
    """))

# Query to verify the updated heights
query = """
    SELECT * FROM multi_event_competitors LIMIT 10;
"""
df = pd.read_sql(query, engine)
print(df)

   person_id  games_id  total_events
0          5         5             2
1          5         6             2
2          5         7             2
3          6         6             4
4          6         7             4
5          7         6             4
6          7         7             4
7          8         8             2
8         15         3             2
9         15        14             2


In [18]:
# Task 3: Identify regions where the average number of medals won per competitor 
# is greater than the overall average. Use subqueries to calculate and compare averages.

df_medals = pd.read_sql("""
    SELECT nr.region_name, COUNT(ce.medal_id) AS total_medals
    FROM competitor_event ce
    JOIN games_competitor gc ON ce.competitor_id = gc.id
    JOIN person_region pr ON gc.person_id = pr.person_id
    JOIN noc_region nr ON pr.region_id = nr.id
    WHERE ce.medal_id IS NOT NULL
    GROUP BY nr.region_name;
""", engine)

df_competitors = pd.read_sql("""
    SELECT nr.region_name, COUNT(DISTINCT gc.person_id) AS total_competitors
    FROM games_competitor gc
    JOIN person_region pr ON gc.person_id = pr.person_id
    JOIN noc_region nr ON pr.region_id = nr.id
    GROUP BY nr.region_name;
""", engine)



# Merge the two DataFrames to align data by 'region_name'
df_merged = pd.merge(df_medals, df_competitors, on='region_name', how='inner')

# Calculate medals per competitor for each region
df_merged['medals_per_competitor'] = df_merged['total_medals'] / df_merged['total_competitors']

# Calculate the overall average medals per competitor
overall_average = df_merged['medals_per_competitor'].mean()

# Filter regions with above-average medals per competitor
df_above_average = df_merged[df_merged['medals_per_competitor'] > overall_average]

df_above_average

,region_name,total_medals,total_competitors,medals_per_competitor
4,Andorra,169,61,2.770492
6,Antigua and Barbuda,132,69,1.913043
7,Argentina,3073,1690,1.818343
10,Australasia,94,51,1.843137
11,Australia,7584,3747,2.024019
...,...,...,...,...
214,Uzbekistan,527,295,1.786441
219,Virgin Islands,290,145,2.000000
221,West Germany,3814,1705,2.236950
222,West Indies Federation,36,13,2.769231


In [21]:
# Task 4: Create a temporary table to track competitors’ participation across different 
# seasons and identify those who have participated in both Summer and Winter games.

with engine.connect() as connection:

    connection.execute(text("DROP TABLE IF EXISTS competitor_seasons;"))

    connection.execute(text("""
    CREATE TEMPORARY TABLE competitor_seasons AS
    SELECT 
        gc.person_id, 
        g.season
    FROM games_competitor gc
    JOIN games g ON gc.games_id = g.id
    GROUP BY gc.person_id, g.season;
"""))
    
    competitors_both_seasons = pd.read_sql("""
    SELECT person_id
    FROM competitor_seasons
    GROUP BY person_id
    HAVING COUNT(DISTINCT season) = 2;
""", connection)
    
competitors_both_seasons

,person_id
0,770
1,4660
2,5429
3,5605
4,7978
...,...
153,129491
154,130148
155,130626
156,131183
